### Лабораторная работа №2

Загрузите данные close_prices.csv. 
В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода между 2013 и 2015 годами.
На загруженных данных обучите преобразование PCA с числом компонент равным 10. 
Скольких компонент хватит, чтобы объяснить 90% дисперсии?

Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.
Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. 
Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?

Какая компания имеет наибольший вес в первой компоненте? Укажите ее название.

In [71]:
import pandas as pd
import numpy as np

#Чтение данных из файла
price = pd.read_csv('close_prices.csv',delimiter=',')

x = price.to_numpy()
x = x[:,1:31].astype(float)

#Метод главных компонет
xc = x - np.mean(x, axis = 0)   #Центрирование данных
xc = xc/np.std(xc, axis = 0)   #Нормирование данных

covMat = np.cov(xc, rowvar = False)

eigVals, eigVecs = np.linalg.eigh(covMat)

#переворачиваем массив собственных значений так, чтобы последний элемент стал первым
s_index = np.arange(eigVals.size-1,0-1,-1)
s_eigVals = eigVals[s_index] 
s_eigVecs = eigVecs[:,s_index]   #переворачиваем массив собственных векторов

n = 10   #Требуемое количество компонент
p_eigVecs = s_eigVecs[:,0:n]
print("Размер массива собственных векторов:".ljust(55), p_eigVecs.shape)

xnew = np.dot(xc, p_eigVecs) #Отображение xc на новый базис
xre = np.dot(xnew, p_eigVecs.transpose())   #возврат на старый базис для первых десяти главных компонент

print("Размер массива исходных данных:".ljust(55), xc.shape)
print("Размер массива данных в новом базисе:".ljust(55), xnew.shape)
print("Размер массива данных, восстановленных в старый базис: ".ljust(55), xre.shape)

nc = 1  #количество главных компонент представляющих 90% информации
while s_eigVals[0:nc].sum()<s_eigVals.sum()*0.9:
    #print(nc, s_eigVals[0:nc].sum())
    nc=nc+1
print() 
print("количество главных компонент, представляющих 90% информации =", nc)


Размер массива собственных векторов:                    (30, 10)
Размер массива исходных данных:                         (374, 30)
Размер массива данных в новом базисе:                   (374, 10)
Размер массива данных, восстановленных в старый базис:  (374, 30)

количество главных компонент, представляющих 90% информации = 6


In [72]:
#Чтение данных из файла про индекс Доу-Джонса
dji = pd.read_csv('djia_index.csv',delimiter=',')
dji = dji.to_numpy()
dji = dji[:,1].astype(float)
dji = dji - np.mean(dji, axis = 0)   #Центрирование данных
dji = dji/np.std(dji, axis = 0)   #Нормирование данных

print("Корреляция между индексом Доу-Джонса и первой компонентой =",np.corrcoef(dji,xnew[:,0])[0,1])

Корреляция между индексом Доу-Джонса и первой компонентой = -0.946848152094537


In [82]:
# Расчёт корреляции цен каждой из компаний со значением первой главной компоненты
cp1=np.zeros(xc.shape[1])

for i in range(xc.shape[1]):
    cp1[i] = abs(np.corrcoef(xc[:,i],xnew[:,0])[0,1])

mn=np.argmax(cp1)

col_names = pd.read_csv('close_prices.csv', nrows=0).columns[1:]
print('Название компании, имеющей наибольший вес в первой компоненте:', col_names[mn])


Название компании, имеющей наибольший вес в первой компоненте: MMM
